In [ ]:
import math
import numpy as np 
import pandas as pd
import warnings
import pandas_datareader.data as web
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.models import Sequential
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Import data

In [ ]:
start_date = "2011-01-1"
end_date = "2021-12-31"
df = web.DataReader(name="AAPL", data_source='yahoo', start=start_date, end=end_date)

In [ ]:
df.head(8)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
features=df.columns
for feature in features:
     print(f'{feature}--->{df[feature].nunique()}')

In [ ]:
history_future_preiod= 60
predication = "Close"
epoch_num = 100
batch = 32

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Aapl close history')
plt.plot(df['Close'])
plt.xlabel('Date')
plt.ylabel('Close price')
plt.show()

In [ ]:
#new data-frame

In [ ]:
df_close=df[['Close']]
df_close=df_close.values
df_close = df_close.astype('float32')
df_close = np.reshape(df_close, (-1, 1))

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
df_close= scaler.fit_transform(df_close)
df_close

In [ ]:
#split data into train and test
train_size = int(len(df_close)* 0.8)
test_size = len(df_close) - train_size

train, test = df_close[0:train_size, :],df_close[train_size:len(df_close), :]

print('Split data into train and test: ', len(train), len(test))

In [ ]:
def features(df, history_time):
    X, Y = [], []
    for i in range(len(df) - history_time - 1):
        window = df[i:(i + history_time), 0]
        X.append(window)
        Y.append(df[i +history_time, 0])
       
        
    return np.array(X), np.array(Y)
X_train, y_train = features(train,history_future_preiod)
X_test, y_test = features(test,history_future_preiod)

In [ ]:
len(X_train)

In [ ]:
X_train[0]

In [ ]:
#put it in 3 dim for lstm
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
X_train.shape

In [ ]:
model = Sequential()

model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, 
               input_shape = (X_train.shape[1], history_future_preiod)))
model.add(Dropout(0.2))
model.add(LSTM(50, activation = 'relu', return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6)
model.compile(loss = 'mean_squared_error',
              #optimizer = 'adam'
              optimizer = opt
             )

In [ ]:
history = model.fit(X_train, y_train,
                    epochs = epoch_num,
                    batch_size = batch,
                    #validation_data = (X_test, y_test),
                    verbose = 1,
                    shuffle = False)


In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.summary()

In [ ]:
#predications

In [ ]:
yp = model.predict(X_test)

In [ ]:
xp = model.predict(X_train)

In [ ]:
yp[:1]

In [ ]:
#unscale predictions and target
train_Predictions = scaler.inverse_transform(xp)
test_Predictions = scaler.inverse_transform(yp)
trainY = scaler.inverse_transform([y_train])
testY = scaler.inverse_transform([y_test])

In [ ]:
#Now reshape
trainPredictions = np.reshape(train_Predictions, newshape = 2154)
testPredictions = np.reshape(test_Predictions, newshape = 493)
y_train = np.reshape(y_train, newshape = 2154)
y_test = np.reshape(y_test, newshape = 493)

In [ ]:
#Root mean square error
from sklearn.metrics import mean_squared_error
train_mse = np.sqrt(mean_squared_error(y_train, train_Predictions))

test_mse = np.sqrt(mean_squared_error(y_test, test_Predictions))
print(f'{mean_squared_error}: {train_mse}')
print(f'{mean_squared_error}: {test_mse}')

In [ ]:
#********the  model result**********

In [ ]:

Y_pre = np.append(trainPredictions, testPredictions)

In [ ]:
model_resul = pd.DataFrame()


model_resul['Predicted_y'] = Y_pre


model_resul.shape

In [ ]:
model_resul.head(5)

In [ ]:
model_resul.plot(figsize=(15,10))
plt.xlabel('Time in days ')
plt.ylabel('Aaple stock')